In [1]:
import urllib
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import csv
import re
import os

In [2]:
def topicCrawl(url):
    
    """Crawls through infinite scroll topic page and grabs the urls for every answer.  
    URL must be in the form 'https://www.quora.com/topic/Irritable-Bowel-Syndrome' to work.  
    Returns list of URLs"""

    linklist = []
    browser = webdriver.Chrome(executable_path='C:\\Users\\jon.driscoll\\Downloads\\chromedriver')
    browser.get(url)
    time.sleep(1)
    elem = browser.find_element_by_tag_name("body")
    no_of_pagedowns = 100
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
        no_of_pagedowns-=1
    post_elems =browser.find_elements_by_xpath('//a[@class="q-box qu-cursor--pointer qu-hover--textDecoration--underline"]')
    for post in post_elems:
        qlink = post.get_attribute("href")
        linklist.append(qlink)
    return linklist

In [3]:
def questionCrawl(url, browser, language = 'en'):
    commentlist = []
    #browser = webdriver.Chrome(executable_path='C:\\Users\\jon.driscoll\\.wdm\\drivers\\chromedriver\\win32\\88.0.4324.96')
    browser.get(url)
    elem = browser.find_element_by_tag_name("body")
    question = url.split('/')[-1].replace('-',' ')
    no_of_pagedowns = 50
    while no_of_pagedowns:
        try:
            elem_more = elem.find_element_by_xpath('//div[text()="Continue Reading"]')
            Hover = ActionChains(browser).move_to_element(elem_more)
            Hover.click(elem_more).perform()
        except:
            pass
        try:
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
            no_of_pagedowns-=1
            elem_more = elem.find_element_by_xpath('//div[text()="Continue Reading"]')
            Hover = ActionChains(browser).move_to_element(elem_more)
            Hover.click(elem_more).perform()
        except:
            pass
    allqtext = browser.find_elements_by_xpath('//p[@class="q-text qu-display--block"]')
    for q in allqtext:
        commentlist.append(q.text)
    df = pd.DataFrame()
    df['Question'] = question
    df['Answer'] = commentlist
    df['URL'] = url
    return df

In [4]:
def searchCrawl(query,browser, no_of_pagedowns = 10, language = 'en'):
    
    """Crawls through infinite scroll topic page and grabs the urls for every answer.  
    URL must be in the form 'https://www.quora.com/search?q=query%20goes%20here to work.  
    Returns list of URLs"""
    
    
    """Quora is available in 4 languages, which you can select with the language argument.  options are:
    'en' for English (default)
    'pt' for Portugese (includes Portugal as well as Brazil)
    'it' for Italian
    'es' for Spanish"""
    query = query.replace(' ', '%20')
    url = f'https://{language}.quora.com/search?q=' + query 
    linklist = []
    #browser = webdriver.Chrome(executable_path='C:\\Users\\jon.driscoll\\.wdm\\drivers\\chromedriver\\win32\\88.0.4324.96')
    browser.get(url)
    time.sleep(1)
    elem = browser.find_element_by_tag_name("body")
    no_of_pagedowns = no_of_pagedowns
    while no_of_pagedowns:
        try:
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)
            no_of_pagedowns-=1
            post_elems =browser.find_elements_by_xpath('//a[@class="q-box qu-display--block qu-cursor--pointer qu-hover--textDecoration--underline Link___StyledBox-t2xg9c-0 roKEj"]')
            for post in post_elems:
                qlink = post.get_attribute("href")
                linklist.append(qlink)
        except:
            pass
    linklist = list(dict.fromkeys(linklist))
    return linklist

In [5]:
def quoraCrawl(query, no_of_pagedowns = 100, language = 'en'):
    browser = webdriver.Chrome(executable_path='C:\\Users\\jon.driscoll\\Downloads\\chromedriver')
    urls = searchCrawl(query, browser, no_of_pagedowns = no_of_pagedowns, language = language)
    dflist = []
    for url in urls:
        df = questionCrawl(url, browser, language = language)
        df['Question'] = url.split('/')[-1].replace('-',' ')
        dflist.append(df)
    dfconcat = pd.concat(dflist)
    dfconcat['Query'] = query
    dfconcat.to_csv(query + '.csv')

In [6]:
keywords = ['ansiedad',
'inquieto',
'insomne',
'insomnio',
'depresión',
'Deprimido',
'preocuparse',
'estrés',
'estresado',
'bienestar',
'salud',
'fatiga',
'salud mental',
'bienestar mental',
'triste',
'tristeza',
'dolor en las articulaciones',
'articulaciones adoloridas',
'energía',
'estado animico',
'estimulante del humor',
'remedio de hierbas',
'suplemento herbario',
'refuerzo de la inmunidad',
'inmunidad',
'el apoyo del estado de ánimo',
'multivitamina',
'colágeno hidrolizado',
]

In [ ]:
for keyword in keywords:
    print(keyword)
    quoraCrawl(keyword, no_of_pagedowns = 10, language = 'en')

ansiedad
